<a href="https://colab.research.google.com/github/spring-board-b2-hate-speech/Group-8/blob/s_work/DL%20model/LSTM_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

X_train = train_df['clean_text']
y_train = train_df['IsHatespeech']
X_test = test_df['clean_text']
y_test = test_df['IsHatespeech']

tfidf_vectorizer = TfidfVectorizer(max_features=2500)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words, lower=True, oov_token='UNK')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_len))
#Do not add the drop out layer after this embedding
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
#Regularization and optimizers
# Early stopping
#

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.005), metrics=['accuracy'])

history = model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_data=(X_test_pad, y_test), verbose=2)

y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")

print("LSTM Performance:")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Precision:",precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))

Epoch 1/5
117/117 - 41s - loss: 0.5349 - accuracy: 0.7277 - val_loss: 0.4937 - val_accuracy: 0.7615 - 41s/epoch - 354ms/step
Epoch 2/5
117/117 - 36s - loss: 0.3593 - accuracy: 0.8433 - val_loss: 0.5476 - val_accuracy: 0.7492 - 36s/epoch - 305ms/step
Epoch 3/5
117/117 - 42s - loss: 0.2619 - accuracy: 0.8920 - val_loss: 0.6272 - val_accuracy: 0.7315 - 42s/epoch - 358ms/step
Epoch 4/5
117/117 - 35s - loss: 0.1921 - accuracy: 0.9225 - val_loss: 0.8584 - val_accuracy: 0.7352 - 35s/epoch - 303ms/step
Epoch 5/5
117/117 - 37s - loss: 0.1319 - accuracy: 0.9473 - val_loss: 0.9407 - val_accuracy: 0.7218 - 37s/epoch - 315ms/step
59/59 [==============================] - 2s 32ms/step
LSTM Performance:
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.66      0.71       944
           1       0.69      0.78      0.74       918

    accuracy                           0.72      1862
   macro avg       0.73      0.72      0.72      1862
weighte